# Parte 2. Feature Selection
Usaremos el metodo de ramdonforest para la selección de características.

In [1]:
import os
import sys

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.insert(0, project_root)

os.chdir(project_root)
print(f"Directorio de trabajo actual cambiado a: {os.getcwd()}")

Directorio de trabajo actual cambiado a: /Users/david.bayter/Documents/Private/challenge-ds-meli


In [2]:
import pandas as pd
import numpy as np
from src.modeling.feature_selection import feature_selection_random_forest
from sklearn.preprocessing import LabelEncoder
from config.config import configs
from src.utils.convert_datatype_utils import set_datatypes

In [3]:
df = pd.read_csv('data/processed/df_to_modeling.csv')
df.dtypes

condition                             int64
base_price                          float64
seller_id                             int64
listing_type_id                      object
price                               float64
buying_mode                          object
category_id                          object
international_delivery_mode          object
accepts_mercadopago                    bool
currency_id                          object
automatic_relist                       bool
status                               object
initial_quantity                      int64
sold_quantity                         int64
available_quantity                    int64
seller_address_state.name            object
seller_address_city.name             object
sub_status_deleted                     bool
sub_status_expired                     bool
sub_status_suspended                   bool
shipping_local_pick_up                 bool
shipping_free_shipping                 bool
shipping_mode                   

In [5]:
df = set_datatypes(df, numericas=configs.col_numericas, categoricas_nom=configs.col_categoricas, booleanas=configs.col_booleanas)
df.dtypes

Columnas numéricas convertidas.
Columnas booleanas convertidas.
Error: La columna 'site_id' no se encontró en el DataFrame.
Columnas categóricas convertidas.


condition                              int64
base_price                           float64
seller_id                           category
listing_type_id                     category
price                                float64
buying_mode                         category
category_id                         category
international_delivery_mode         category
accepts_mercadopago                     bool
currency_id                         category
automatic_relist                        bool
status                              category
initial_quantity                       int64
sold_quantity                          int64
available_quantity                     int64
seller_address_state.name           category
seller_address_city.name            category
sub_status_deleted                      bool
sub_status_expired                      bool
sub_status_suspended                    bool
shipping_local_pick_up                  bool
shipping_free_shipping                  bool
shipping_m

In [13]:
selected_features, importance_df, rf_model = feature_selection_random_forest(
        df_label_encoded,
        target_col='condition',
        n_features=15,
        n_estimators=600,
        max_depth=15
    )

Iniciando la selección de características con One-Hot Encoding...
El número de características después de la codificación es: 71
Entrenando el modelo Random Forest...

Accuracy en entrenamiento: 0.8831
Accuracy en test: 0.8614

Top 15 features seleccionadas:
                          feature  importance
33           listing_type_id_free    0.202333
6                initial_quantity    0.185549
8              available_quantity    0.167715
7                   sold_quantity    0.060708
2                           price    0.060432
0                      base_price    0.058747
3                     category_id    0.034328
32               title_cont_nuevo    0.021616
9        seller_address_city.name    0.018423
1                       seller_id    0.017884
29                 warranty_group    0.017231
23  payment_group_Tarjeta_Credito    0.014158
38         listing_type_id_silver    0.013410
31               title_cont_usado    0.013236
5                automatic_relist    0.009986
